In [20]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
import torch

# model = GPT2LMHeadModel.from_pretrained('eunjin/kogpt2-finetuned-wellness')
model = GPT2LMHeadModel.from_pretrained('jack-oh/KoGPT2_finetuned_wellness')
# model = GPT2LMHeadModel.from_pretrained('/home/jack/project/github/WellnessChatbot/finetuned_model')

U_TKN = '<usr>'
S_TKN = '<sys>'
EOS = '</s>'
MASK = '<unused0>'
PAD = '<pad>'
SENT = '<unused1>'

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 


# max_length = 50, num_beams = 10, do_sample = False, top_k = 50, no_repeat_ngram_size = 2, temperature = 0.85
def chat(utterance,maxlen=50,numbeams=10,sampling=True,topk=50,ngramsize=2,temp=0.85):
    print(f'나: {utterance}\n')
    
    with torch.no_grad():
            qs=[]
            q = utterance
            qs.append(q) # history 저장

            a=''
            user = U_TKN + q + SENT + a
            encoded = tokenizer.encode(user)
            input_ids = torch.LongTensor(encoded).unsqueeze(dim=0)
            output = model.generate(input_ids,max_length=maxlen,
                                         num_beams=numbeams, do_sample=sampling, 
                                         top_k=topk, no_repeat_ngram_size=ngramsize,
                                        temperature=temp)
            a=tokenizer.decode(output[0])
            idx = torch.where(output[0]==tokenizer.encode('<sys>')[0])
            chatbot = tokenizer.decode(output[0][int(idx[0])+1:], skip_special_tokens=True)
 
            if '답변' in a: # 응, 아니 등이 input으로 들어왔을 때
                a_new = ''
                user = U_TKN + ''.join(qs[-2:]) + SENT + a_new # 직전 history 가지고 와서 sentiment 고려해주기
                encoded = tokenizer.encode(user)
                input_ids = torch.LongTensor(encoded).unsqueeze(dim=0)
                output = model.generate(input_ids,max_length=maxlen,
                                         num_beams=numbeams, do_sample=sampling, 
                                         top_k=topk, no_repeat_ngram_size=ngramsize,
                                        temperature=temp)
                a_new = tokenizer.decode(output[0], skip_special_tokens=True)
                idx = torch.where(output[0]==tokenizer.encode('<sys>')[0])
                chatbot = tokenizer.decode(output[0][int(idx[0])+1:], skip_special_tokens=True)
                
                result = chatbot.strip()
            
            else: 
                result = chatbot.strip()
                
    print(f"챗봇: {result}\n")
    print('='*100)

    return result



In [9]:
# 'jack-oh/KoGPT2_finetuned_wellness'
while 1:
    sent = input()
    chat(sent)

나: 고민이 많다

챗봇: 마음이 예민해져서 매우 괴로우셨겠어요. 저랑 차분하게 잠시 대화를 해봐요.

나: 공부하는게 힘들어

챗봇: 공부하면 더 많은 선택을 할 수 있죠.

나: 그치만 너무 힘든걸

챗봇: 조금만 더 버텨보세요.

나: 싫어 너무 무기력해

챗봇: 무기력함 때문에 힘든 시기를 보내고 계시는군요. 저도 마음이 안 좋아요.

나: 응 아무것도 하기 싫어

챗봇: 00님이 힘든 것 같아 걱정이에요. 더 얘기해볼까요?

나: 성적이 자꾸 떨어져

챗봇: 공부한 만큼 나올 거예요.

나: 친구들이랑도 잘 안맞는것같아

챗봇: 잘하는 건 칭찬해주고 본받을 점은 본받으면서 그대로 사랑해주세요.

나: 요새 육아하느라 지쳐서 너무 우울해

챗봇: 00님이 울적하시다니 저도 마음이 안 좋아요.

나: 남편이 집안일을 안도와주는데 어떻게 해야돼?

챗봇: 잘 분담해보세요.

나: 회사 출근하기 너무 싫다

챗봇: 회사에는 동료가 있을 뿐이에요.

나: 일이 손에 잘 안잡혀

챗봇: 힘들겠지만 하나씩 해보세요.



KeyboardInterrupt: Interrupted by user

In [24]:
chat('친구들이 자꾸 차갑게 말 하니까 대화하기 싫어서 피하게 되더라')

나: 친구들이 자꾸 차갑게 말 하니까 대화하기 싫어서 피하게 되더라

챗봇: 내 인생은 실패작이라고 자존감이 싶어요.



'내 인생은 실패작이라고 자존감이 싶어요.'